#### pandas data 호출

In [18]:
import numpy as np
import pandas as pd

In [66]:
path = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/train.csv'
path2 = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/test.csv'
path3 = 'C:/Users/tmznq/workspace/ml_scratch/dataset/titanic/gender_submission.csv'
df = pd.read_csv(path)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

df3.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


### indexing 참고할 것.
df[0] : 점색인. 열
df[:7] : 연속색인. 행
df[['col1', 'col2']] : 열색인, boolean 색인

#### merge, join, concat 복습

#### 1. removing duplicates
- 중복된 row를 식별하고 이를 제거하는 방법을 학습한다.
- df.duplicated() : df.drop_duplicates()의 하위 개념으로 함수 적용 시, 중복인지 아닌지 여부를 boolean Series로 반환.
- subset : column label 혹은 sequence of labels. 중복을 식별할 열을 지정.
- keep
- inplace

In [ ]:
df.drop_duplicates = df[~df.duplicated()] : 같은 의미. why? boolean Series에 의한 indexing

In [ ]:
df.drop_duplicates(['k1']) # 1개의 열을 고르더라도 subset으로 다루기 때문에 여러 열을 사용하듯이 지정해야함.

#### 2. transforming, mapping

#### mapping
- 'A' 열의 설명을 위한 새로운 column을 추가하려고 할 때.
- 'A'의 각 열의 key을 값으로하는 열을 추가하고 싶을 때. map 함수 사용
- map : vector(Series)의 각 성분별 함수 결과 또는 각 성분을 key로하는 value를 리턴한다. df에서는 사용불가

ex) data['col1'] = data['A'].map(str, lower).map(dict) # dict의 key는 'A'의 각 열과 동일하다.

#### replace value
- 결측치 대체
- 

In [1]:
data = Series([1, -999, 3, -999, 5])
data

NameError: name 'Series' is not defined

In [ ]:
data.replace(-999, np.nan)
data.replace([-999, 5], [np.nan, 0])

#### renaming axis indexes

#### Discretization and binning(개별화와 양자화)
연속된 data는 종종:
- 개별로 분할하거나
- 분석을 위한 그룹(bin)으로 나눈다.

pd.cut()
- bin으로 나눌 구간은 ( , ]으로 정의되므로 첫 항은 빠진다. 따라서 일반적으로 +- 5% 정도 범위를 늘려서 계산한다.

#### 6. filtering outliers
- df.describe()

In [ ]:
# 통계량 boolean indexing 적용해보기
ex) data[['col1', 'col2']][:5][True, True, False, False, False] 

In [ ]:
# Series 적용해보기
col = data[3]
col[np.abs(col) > 3] 

- df.any

In [4]:
arr = np.array([0, -5, 2, 7])
arr > 0

array([False, False,  True,  True])

In [ ]:
l = [0, -5, 2, 7]

b = list(map(lambda x: x>0, l)) # list boolean은 이런식으로...?
any(b) # 1개라도 True면 True

In [12]:
all(b) # 전부 True가 아니라면 False

False

In [ ]:
(np.abs(data) > 3).any(axis = 0) # np.abs(data) > 3는 df으로 boolean indexing 되서 반환된다.

In [ ]:
data[(np.abs(data) > 3).any(axis = 0)] # boolean indexing으로 True인 데이터만 df으로 반환.

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3 # 왼쪽의 data는 data size가 유지됨.
data.describe()

#### 7. Permutation and random sampling(치환과 임의 샘플링)
비복원 추출  
np.permutation()
- 중복된 row는 없음.
- 주머니에서 'a'구슬을 뽑으면 그 구슬은 다시 집어넣지 않는다.
- row를 임의의 순서로 재배열하거나, 임의 샘플링하는 방법을 알아본다.
- 순서를 바꾸고 싶은 만큼의 길이를 np.permutation 함수에 넘기면 임의 순서의 정수배열을 얻을 수 있다.

In [6]:
df = DataFrame(np.arange(5 * 4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

NameError: name 'DataFrame' is not defined

In [20]:
# 여기서 반만 train data 나머지는 test data로 쓰려고 한다.
# 주로 사용하는 예시
# titanic data 다 합친거를 다시 7대3으로 나눠보자.
i = sampler.size * 0.8
i = round(i)
train_idx = sampler[:i]
val_idx = sampler[i:]
print(train_idx)
print(val_idx)

[4 0]
[3 1 2]


In [ ]:
train_df = df.iloc[train_idx]
val_df = df.iloc[val_idx]

In [23]:
df.take(sampler) # sampler에 의해 섞인 df 

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11


In [25]:
df.iloc[sampler] # 위와 동일한 연산

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11


복원추출
- 중복된 row도 포함.
- np.random.randint 사용.
- 주머니에서 구슬뽑고 다시 구슬 넣음

np.random.randint()
- [low, high] 사이의 임의의 정수 size만큼 리턴

#### 8. Computing indicator and dummy variables(표시자와 범주형 변수)
- 예측 model에는 전부 숫자만 들어갈 수 있음.
- 이항 : 남녀, P/N, 동전 앞/뒤
- 다항 : 서열(범주+순서에 의미가 있는 것), Categorical

one-hot encode
- sparse하다? 대부분이 0이고 1이 드물다.
- sparse matrix : df를 조작하여 만들고, one-hot encoding하여 숫자로 풀어서 다시 만든 것.
- 예측 model에는 pandas로 조작한 df가 아니라 one-hot encoding하여 전부 숫자로 변환한 것을 넣는다.
- 정보가 유실되지 않는다.

서열범주
- one-hot encoding : data가 유실되지 않음. but, data가 무거워짐.
- 등간 숫자 : 의미 있는 순서를 1,2,3으로 만듬(숫자의 weight를 조절할 수 있음. ex) 10, 30, 50. / but, data가 유실될 수도 있음

Categorical
- ex) 고기의 육류별 종류를 육해공으로 압축.
- 분석 목적에 따라 압축시키는 방법은 전부 다름.

#### pd.get_dummies()
- 범수형 변수(지시자)를 one-hot encoding(더미변수)형식으로 변경하는 것이다.
- drop_first : 한 차원을 더 축소시키는 방법. 주로 R에서 사용
- add_prefix('name_') : 더미 이름 붙여주기

In [ ]:
# df + dummies 
pd.get_dummie(df, key = ['key'])

#### movielens 영화 데이터의 더미변수 생성
- 구분자가 2글자 이상이거나, 정규 표현식인 경우. c parsing engine이 동작하지 않아 경고메세지 나옴. engine=python으로 해주셈

In [ ]:
# 장르를 통해 one-hot encoding을 실시해보자.

genre_iter = (set(x.split('|')) for x in movies.genres) # 장르 데이터를 집합으로 만듦. 
# 이는 generator로써 값을 가진게 아니라 iterator를 동작하게 하는 역할
genre_iter # 데이터를 담고 있지 않음.

In [27]:
# 범주를 나눌 때 효과적인 code
genre = sorted(set.union(*genre_iter))

- set.union 함수는 genre_iter을 매 반복마다 합집합 시킴. 즉, data를 늘리지않고 unique한 요소들만 계속 추가시킴.
- sorted 내장함수는 iterable의 각 속성을 정렬해서 list로 반환한다.

___
- generator, iterator, iterable에 대해 알아보자.

f(v) for v in container : iterator. 값을 가진게 아니라 일정하게 찍어내기만함. 불변(immutable)
- list comprehension : [f(v) for v in container]
- generator : (f(v) for v in container), tuple처럼 보이지만 generator ^ ^
- set : {f(v) for v in container}
- dict : {}

___

In [ ]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)

In [ ]:
for i, gen in enumerate(movies, genres):
    dummies.loc[i, gen.split('|')] = 1

- df에서 genre 열을 삭제하고, dummies로 만든 새로운 column을 결합하자.

## 문자열 다루기

In [84]:
# 정규표현식
# 자주 까먹음
# 책 미리 보고 올 것.

#### 과제 1 : titanic 데이터에서 'Age' column을 bin으로 나누기
- label = ['청소년', '청년', '중년', '장년']
- bins = 3

In [85]:
Age_2 = pd.cut(df['Age'], bins = 4, labels = ['청소년', '청년', '중년', '장년'])

TypeError: '<=' not supported between instances of 'str' and 'float'

In [86]:
df['Age'] = Age_2
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,청년,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,청년,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,청년,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,청년,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,청년,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,중년,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,청소년,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,청년,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,청소년,1,0,237736,30.0708,NaN,C


#### 과제 2 : titanic 데이터 random sampling으로 다시 train, test 2개로 나누기
- np.permutation() 혹은 np.random.randint() 사용
- train 대 test의 비율은 8 대 2

In [87]:
df.shape

(891, 12)

In [88]:
sampler = np.random.permutation(20)
sampler

array([12,  8,  1, 17, 13,  9,  5,  3,  7, 18, 15,  4,  6, 19,  0, 11, 10,
       14,  2, 16])

In [89]:
i = round(sampler.size * 0.8)
train_idx = sampler[:i]
test_idx = sampler[i:]

In [90]:
train_idx = df.iloc[train_idx]
train_idx

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
12,13,0,3,"Saundercock, Mr. William Henry",male,청소년,0,0,A/5. 2151,8.0500,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,청년,0,2,347742,11.1333,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,청년,1,0,PC 17599,71.2833,C85,C
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
13,14,0,3,"Andersson, Mr. Anders Johan",male,청년,1,5,347082,31.2750,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,청소년,1,0,237736,30.0708,NaN,C
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,청년,1,0,113803,53.1000,C123,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,청소년,3,1,349909,21.0750,NaN,S
18,19,0,3,"Vander Planke, Mrs. Julius (Emelia Maria Vande...",female,청년,1,0,345763,18.0000,NaN,S


In [91]:
test_idx = df.iloc[test_idx]
test_idx

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,청소년,1,1,PP 9549,16.7000,G6,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,청소년,0,0,350406,7.8542,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,청년,0,0,STON/O2. 3101282,7.9250,NaN,S
16,17,0,3,"Rice, Master. Eugene",male,청소년,4,1,382652,29.1250,NaN,Q
